Make subtitle as properties of title

In [1]:
import json
import copy
import re

In [75]:
s = "1. What are preferences over prospects?"
idx_toc_dict = {}
idx = s.split(' ')[0].strip()
toc_title = "".join(s.split(idx)[1:]).strip()
# order matters
if idx[-1] == '.':
    idx = idx[:-1]
idx_toc_dict[toc_title] = 0
idx_toc_dict

s = "7.1.Sortals for infants"
# ERROR: how to tolerate additional dots 7.1.Sortals for infants
last_digit = re.findall(r'\d+', s)[-1]
s[:s.index(last_digit)+1]

'7.1'

## Manually fix

 this entry toc https://plato.stanford.edu/entries/capability-approach/
 
 Also

- 2.Living Stoically
- 2.The Justification of Revelatory Claims
- 2.Challenges and Replies
- 2.Definitions of Disability
- perception-auditory
    3.2.2 Pitch, Timbre, and Loudness
    3.2.3 Is Pitch Physical?
    3.3.4 Disanalogies with Color
    3.3.5 Analogies with Color
- ?
    4.2.3 The Contents of Speech Perception
    4.3.4 Is Speech Perception Auditory?


In [83]:
with open('get_data/manually_fixed/ent_meta_list_fixed.json', 'r') as f:
    ent_list = json.load(f)
print("Length of ent_list:", len(ent_list))

# biuld a dictionary
ent_toc_dict = {}

for item in ent_list:
    ent_url = item['url']
    # get toc
    toc_fixed_list, toc_list = [], []
    # ERROR: there are nested toc for "Supplementary Document"
    # e.g., https://plato.stanford.edu/entries/analysis/
    toc_ori_list = [list(i.values())[0] for i in item['toc'] if ".html" not in list(i.keys())[0]]
    # ignore the academic tool, bibliographies, etc.
    for s in toc_ori_list:
        if s == "Bibliography":
            break
        toc_list.append(s.strip())
        

    # ERROR: some times there is no index (223 in total)
    for i in range(len(toc_list)):
        s = toc_list[i]
        if not s[0].isdigit():
            # if it is the first one (nothing ahead of it), add 1
            if i == 0:
                fixed_s = "1." + " " + s

            for j in range(i, -1, -1):
                if toc_list[j][0].isdigit():
                    # add index
                    cat_idx = toc_list[j].split(' ')[0].strip()
                    if cat_idx[-1] == '.':
                        cat_idx = cat_idx[:-1]
                    add_idx = cat_idx + "." + str(i-j)
                    fixed_s = add_idx + " " + s
                    break
            toc_fixed_list.append(fixed_s)
            # DON'T toc_list[i] = s, because we don't want to change the list
            # print("\n",s)
            # print(json.dumps(toc_list, indent=4))
        else:
            toc_fixed_list.append(s)
    # print(toc_fixed_list)

    # define hierarchy
    idx_toc_dict = {}
    toc_nest_dict = {}
    toc_concat_list = []
    # build idx_toc_dict

    for s in toc_fixed_list:
        idx = s.split(' ')[0].strip()
        toc_title = "".join(s.split(idx)[1:]).strip()
        # order matters
        if idx[-1] == '.':
            idx = idx[:-1]
        idx_toc_dict[idx] = toc_title
        # print(idx, idx_toc_dict[idx])

    # build toc_nest_dict
    for s in toc_fixed_list:
        # ERROR: how to tolerate additional dots 7.1.Sortals for infants
        
        idx = s.split(' ')[0].strip()
        toc_title = "".join(s.split(idx)[1:]).strip()
        idx_sep = [str(i) for i in idx.split('.') if i and i.isdigit()]
        if len(idx_sep) == 1: # 1. Organ Retrieval from the Dead
            toc_nest_dict[toc_title] = {}
            toc_concat_list.append(toc_title)
        elif len(idx_sep) == 2: # 1.2 Proposals for reform
            pre_idx = idx_sep[0]
            try:
                pre_cat = idx_toc_dict[pre_idx]
                toc_nest_dict[pre_cat][toc_title] = {}
                toc_concat_list.append(pre_cat + ": " + toc_title)
            except:
                print("ERROR:", idx_sep, "\n",idx_toc_dict, "\n========")

        elif len(idx_sep) == 3: # 1.2.2 End the family\u2019s power of veto
            pre_idx = idx_sep[0] + '.' + idx_sep[1]
            try:
                pre_cat = idx_toc_dict[pre_idx]
                pre_pre_cat = idx_toc_dict[pre_idx.split('.')[0]]
                toc_nest_dict[pre_pre_cat][pre_cat][toc_title] = {}
                toc_concat_list.append(pre_pre_cat + ": " + pre_cat + ": " + toc_title)
            except:
                print("ERROR:", idx_sep, "\n",idx_toc_dict, "\n========" )
    # print(toc_concat_list)

    # update dict
    ent_toc_dict[ent_url] = {
        'title': item['title'],
        'toc_list': toc_fixed_list,
        'toc_nest_dict': toc_nest_dict,
        'toc_concat_list': toc_concat_list
    }
    # print(json.dumps(ent_toc_dict[ent_url], indent=4))

# save 
with open('get_data/toc/ent_toc_dict.json', 'w') as f:
    json.dump(ent_toc_dict, f, indent=4)

Length of ent_list: 1769


# use related entries as labels

In [ ]:
# open 'get_data/toc/ent_toc_dict.json'
with open('get_data/toc/ent_toc_dict.json', 'r') as f:
    ent_toc_dict = json.load(f)
# print all the titles
for k, v in ent_toc_dict.items():
    print(v['title'])
    print(v['toc_concat_list'])


In [23]:
# read entry_meta_dict
with open('database/entry_meta_dict.json', 'r') as f:
    entry_meta_dict = json.load(f)

# print all related entries
for ent_dict in list(entry_meta_dict.values()):
    # print(ent_dict)
    # if not ent_dict:
    #     print("ERROR: empty ent_dict", ent_dict)
    rel_ent_list = ent_dict['related_entries']
    # print(rel_ent_list['entry_list'])
    ent_title_list = rel_ent_list['entry_list']
    # for each related entry, label the type of the entry
    for ent_title in ent_title_list:
        for k, v in entry_meta_dict.items():
            if ent_title == v['title']:
                print(v['ent_type'], "|", ent_title)
                


thinker | William of Champeaux
thinker | Plato
thinker | Voltaire
topic | Latin American Philosophy
topic | Darwinism
thought of thinker | Bayes’ Theorem
topic | Latin American Philosophy
thinker | Giles of Rome
thinker | Godfrey of Fontaines
thinker | Henry of Ghent
topic | Latin American Philosophy
thinker | Ammonius
topic | Neoplatonism
thinker | Plotinus
thinker | Proclus
thinker | Syrianus
topic | Africana Philosophy
topic | Négritude
thinker | al-Farabi
thinker | al-Kindi
thinker | Alexander of Aphrodisias
thinker | Aristotle
thinker | Ibn Rushd [Averroes]
thinker | Ibn Sina [Avicenna]
thinker | Olympiodorus
thinker | Plato
thinker | Plotinus
thinker | Proclus
thinker | Radulphus Brito
thinker | Gregory of Rimini
thinker | Zeno of Elea
topic | Lvov-Warsaw School
thinker | al-Farabi
thinker | al-Ghazali
thinker | Ibn ‘Arabî
thinker | Ibn Sina [Avicenna]
thinker | Ikhwân al-Safâ’
thinker | Mulla Sadra
thinker | Suhrawardi
thinker | Archytas
thinker | Aristotle
thinker | Heraclitus


# extract labels from tables of content